In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 2.9MB/s 


In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook



/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [3]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix_day1/matrix_day3"

/content/drive/My Drive/Colab Notebooks/dw_matrix_day1/matrix_day3


In [4]:
ls

data/                 DW_matrix_day4_meta.ipynb
DW_matrix_day3.ipynb  DW_matrix_day4_roboczy.ipynb


In [5]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)
df.shape

(18280, 48)

In [0]:

def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[feats].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)
   

In [7]:
df['brand_cat'] = df['brand'].factorize()[0]
run_model(['brand_cat'])

(-58.38655694633361, 4.223555478221712)

In [8]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.47223572384038, 4.328288468270897)

In [9]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [10]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [11]:
df.head()

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,reviews,sizes,skus,sourceurls,upc,vin,websiteids,weight,brand_cat
0,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",39.89,39.89,NaN,NaN,NaN,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,True,NaN,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
1,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",51.99,51.99,NaN,NaN,new,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,False,UnbeatableSale - Walmart.com,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
2,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,40.02,40.02,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2016-03-08T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
3,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,50.31,50.31,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2015-11-30T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
4,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""

In [12]:
test = {'key': 'value'}
test['key']

str(test)

"{'key': 'value'}"

In [13]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'


literal_eval(str_dict)[0]['value'][0]

'Men'

In [0]:

def parse_features(x):
   output_dict = {}
   if str(x) == 'nan': return output_dict
   
   features = literal_eval(x.replace('\\"', '"'))
   for item in features:
     key = item['key'].lower().strip()
     value = item['value'][0].lower().strip()
 
     output_dict[key] = value
 
   return output_dict
 
df['features_parsed'] = df['features'].map(parse_features)

In [15]:
df['features_parsed'].head()

0    {'gender': 'men', 'shoe size': 'm', 'shoe cate...
1    {'gender': 'men', 'shoe size': 'm', 'shoe cate...
2    {'gender': 'men', 'color': 'black', 'shipping ...
3    {'gender': 'men', 'color': 'black', 'shipping ...
4    {'gender': 'men', 'color': 'black', 'shipping ...
Name: features_parsed, dtype: object

In [17]:
keys = set()

df['features_parsed'].map( lambda x: keys.update(x.keys()) )

len(keys)

476

In [27]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)


In [28]:
df.columns


Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_lining', 'feat_applicable', 'feat_material type', 'feat_bezel',
       'feat_ring size', 'feat_age start', 'feat_use', 'feat_removable hood',
       'feat_sub style', 'feat_quantity in set'],
      dtype='object', length=1002)

In [59]:
df[ df['feat_brand'].isnull() ].shape

(9391, 1002)

In [31]:
df.shape[0]

18280

In [33]:
df[ False == df['feat_age start'].isnull() ].shape[0] / df.shape[0] * 100

0.4431072210065646

In [34]:
df[ False == df['feat_athlete'].isnull() ].shape[0] / df.shape[0] * 100

0.0437636761487965

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[ False == df[get_name_feat(key)].isnull() ].shape[0] / df.shape[0] * 100

In [42]:
#keys_stat
{k:v for k,v in keys_stat.items() if v> 30}


{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].map(lambda x: str(x).lower()).factorize()[0]
df['feat_color_cat'] = df['feat_color'].map(lambda x: str(x).lower()).factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].map(lambda x: str(x).lower()).factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].map(lambda x: str(x).lower()).factorize()[0]
df['feat_material_cat'] = df['feat_material'].map(lambda x: str(x).lower()).factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].map(lambda x: str(x).lower()).factorize()[0]
df['feat_style_cat'] = df['feat_style'].map(lambda x: str(x).lower()).factorize()[0]

for key in keys:
  df[get_name_feat(key)+'_cat'] = df[get_name_feat(key)].factorize()[0]

  

In [106]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())
#df[df.brand == df.feat_brand][['brand', 'feat_brand']].shape
df[df.brand == df.feat_brand].head()

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,...,feat_charger included_cat,feat_materials_cat,feat_hooded_cat,feat_country of origin_cat,feat_number of wheels_cat,feat_made in_cat,feat_country/ organization_cat,feat_net weight_cat,feat_finish_cat,feat_style name_cat,feat_insulation_cat,feat_boxed-product dimensions_cat,feat_display technology_cat,feat_shoe fastening type_cat,feat_heart rate monitor_cat,feat_leather grade_cat,feat_looks best on these face shapes:_cat,feat_age group_cat,feat_place of origin_cat,feat_golf shoe type_cat,feat_number of batteries included_cat,feat_mirrored_cat,feat_age segment_cat,feat_uv rating_cat,feat_isbn_cat,feat_100% adidas authentic_cat,feat_is wheeled_cat,feat_instrument_cat,feat_date first available at amazon.ca_cat,feat_smart watch_cat,feat_lining_cat,feat_applicable_cat,feat_material type_cat,feat_bezel_cat,feat_ring size_cat,feat_age start_cat,feat_use_cat,feat_removable hood_cat,feat_sub style_cat,feat_quantity in set_cat
0,AVpfHrJ6ilAPnD_xVXOI,NaN,josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",39.89,39.89,NaN,NaN,NaN,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,True,NaN,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,AVpfHrJ6ilAPnD_xVXOI,NaN,josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",51.99,51.99,NaN,NaN,new,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,False,UnbeatableSale - Walmart.com,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,AVpfHsWP1cnluZ0-eVZ7,NaN,servus by honeywell,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,40.02,40.02,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2016-03-08T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,AVpfHsWP1cnluZ0-eVZ7,NaN,servus by honeywell,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":["

In [84]:
#feats = ['']
df['feat_brand_cat'].head()

0    0
1    0
2    1
3    1
4    1
Name: feat_brand_cat, dtype: int64

In [107]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'], model)

(-57.31772737308895, 4.148148368170299)

In [112]:
feats_cat = [x for x in df.columns if 'cat' in x]
feats_cat

['categories',
 'brand_cat',
 'featlocation - city/state',
 'featrecommended location',
 'featclothing category',
 'featcatalog',
 'featlocation - country',
 'featshoe category',
 'featmulti pack indicator',
 'featfabrication',
 'featcertifications and listings',
 'feat_location - city/state',
 'feat_recommended location',
 'feat_clothing category',
 'feat_catalog',
 'feat_location - country',
 'feat_shoe category',
 'feat_multi pack indicator',
 'feat_fabrication',
 'feat_certifications and listings',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_combined shipping_cat',
 'feat_antiscratch lens coating_cat',
 'feat_autographed_cat',
 'feat_atpv arc rating_cat',
 'feat_adjustable_cat',
 'feat_closure style_cat',
 'feat_packageweight_cat',
 'feat_lens technology_cat',
 'feat_inv_cat',
 'feat_type of sole_cat',
 'feat_chain length_cat',
 'feat_compatible devices_cat',
 'fea

In [0]:
#feats = ['brand_cat','feat_brand_cat', 'feat_color_cat', 'feat_gender_cat', 'feat_manufacturer part number_cat', 'feat_material_cat' ]
feats = ['brand_cat','feat_metal type_cat','feat_shape_cat','feat_style_cat','feat_brand_cat','feat_ring size_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_sport_cat', 'feat_style_cat' ]
#feats += feats_cat

#feats = list(set(feats))

#feats += ['feat_full product manual_cat', 'feat_parts_cat',]

model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats, model)

In [142]:
X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X,y);
eli5.show_weights(perm, feature_names=feats)



(-57.299806382502176, 4.329131479599527)


Weight,Feature
0.2579 ± 0.0071,brand_cat
0.1008 ± 0.0083,feat_material_cat
0.0226 ± 0.0026,feat_gender_cat
0.0177 ± 0.0011,feat_brand_cat
0.0127 ± 0.0012,feat_shape_cat
0.0065 ± 0.0021,feat_metal type_cat
0.0036 ± 0.0009,feat_ring size_cat
0.0016 ± 0.0005,feat_style_cat
0.0010 ± 0.0002,feat_style_cat
0.0002 ± 0.0000,feat_sport_cat


In [94]:
df['brand'].value_counts(normalize=True)

nike                 0.097210
puma                 0.033315
ralph lauren         0.028775
vans                 0.021116
new balance          0.020295
                       ...   
poc                  0.000055
shoe goo             0.000055
red wing heritage    0.000055
big smith            0.000055
chopard              0.000055
Name: brand, Length: 1732, dtype: float64

In [102]:
#df[df['brand'] == 'nike'].features_parsed.head().values
df[df['brand'] == 'nike'].features_parsed.sample(5).values

array([{'gender': 'boys', 'shoe category': 'boys shoes', 'color': 'lite crimson/white/black', 'casual & dress shoe style': 'athletic', 'manufacturer part number': '599812602', 'brand': 'nike', 'shoe width': 'medium'},
       {'style': 'athletic sneakers', 'condition': 'new without box', 'shade': 'hot lava-teal', 'type 2': 'tennis'},
       {'sport': 'soccer', 'main color': 'green', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'running shoes', 'occasion': 'running shoes', 'material': 'synthetic-and-mesh', 'gender': 'men', 'shoe size': '9 us men', 'shoe category': "men's shoes", 'assembled product dimensions (l x w x h)': '12.00 x 8.00 x 6.00 inches', 'fabric content': 'synthetic-and-mesh', 'color': 'beige', 'model': '704889 013', 'shoe closure': 'lace-up', 'casual & dress shoe style': 'running shoes', 'manufacturer part number': '704889 013', 'variant group id': '224#mp#704889 013', 'brand': 'nike', 'age group': 'men', 'shoe width': 'd(m)'},
       {'sport': 'socc

In [104]:
df['feat_age group'].value_counts()

adult               4563
men                  350
child                 77
men's                 33
unisex                 6
infant                 4
toddler                4
mens                   4
boys'                  3
youth                  2
men||women             2
women                  2
women ,�� unisex       2
12 up                  1
adult ,�� teen         1
Name: feat_age group, dtype: int64